Key notes: Please refer to "https://github.com/ganamir/MLDAAPP" for more information!

WARNING! Keep in mind to run these scripts on a cloud service such as Google Colab!
Unless you possess server level computational resources you may overheat and damage your computer parts! Machine Learning based Computer Vision is a far more computationally intensive process than a typical computer vision program.

# **Install & Import all of dependancies and functions:**

In [ ]:
#Connect Google Drive for ease of use when transfering/uploading files:
from google.colab import drive
drive.mount('/content/drive')

#Download & Upload Package into the local environment:
!pip install ultralytics
!pip install roboflow
!pip install torch
!pip install torchvision
!pip install av
!pip install ffmpeg
!pip install supervision
!pip install PIL
#!pip install certifi
#!pip install cycler
!pip install pyparsing
!pip install pyheif
import pyheif
import csv
from moviepy.editor import *
import supervision as sv
import math
import ultralytics
from ultralytics import YOLO
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image
import matplotlib.pyplot as plt
import torchvision
import cv2
import ffmpeg
import av
import av.datasets
from google.colab.patches import cv2_imshow
from IPython import display
import pandas as pd
import numpy as np
import numpy.ma as ma
import random
from numpy import loadtxt
from numpy import asarray
from numpy import save
from pandas.io.parsers.readers import RangeIndex
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
!git clone https://github.com/ultralytics/ultralytics.git
display.clear_output()

#Functions:
def video_editing(input, t0, t1, fps, spd, w):
  #Video Modification:
  SOURCE_VIDEO_PATH = input
  #Modify Video Length, Framerate, & Speed:
  clip1 = VideoFileClip(SOURCE_VIDEO_PATH)
  clip1 = clip1.subclip(t0,t1)
  clip1_resol = clip1.resize(width = w)
  new_clip2 = clip1_resol.set_fps(fps)
  new_clip3 = new_clip2.fx(vfx.speedx, spd)
  #Save the File:
  file_path = os.path.dirname(SOURCE_VIDEO_PATH)
  file_name = os.path.basename(SOURCE_VIDEO_PATH)
  file_name = os.path.splitext(file_name)[0]
  starting_value = 1
  while True:
    folder_name = file_name + str(starting_value)
    folder_path = os.path.join(file_path, folder_name)
    if not os.path.exists(folder_path):
      os.makedirs(folder_path)
      break
    starting_value += 1
  new_clip3.write_videofile(folder_path+"/"+file_name+".mp4" , codec = "libx264")
  global global_video
  global_video = folder_path+"/"+file_name+".mp4"

def save_to_csv(filename, column_name, data):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([column_name])
        for value in data:
            writer.writerow([value])

def draw_dots(img,x,y):
  %matplotlib inline
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  plt.imshow(img)
  plt.plot(x, y, marker = 'o', markersize = 2)

def draw_lines(img,x1,y1,x2,y2):
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  plt.imshow(img)
  xlp, ylp = [x1,x2],[y1,y2]
  plt.plot(xlp,ylp,marker = 'o', markersize = 2)

def draw_polygons(img,x1,y1,x2,y2,x3,y3,x4,y4):
  %matplotlib inline
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  plt.imshow(img)

  coords1 = [[x1,y1],[x2,y2],[x3,y3],[x4,y4]]
  coords1.append(coords1[0])
  xs, ys = zip(*coords1)
  for x,y in zip(xs,ys):
    label = "{:.2f}".format(y)
    plt.annotate(label,
                (x,y),
                textcoords="offset points",
                xytext = (0,10),
                ha ='center')
  plt.plot(xs,ys)

def area_by_shoelace(x, y):
  "Assumes x,y points go around the polygon in one direction"
  return abs( sum(i * j for i, j in zip(x,y[1:] + y[:1]))
               -sum(i * j for i, j in zip(x[1:] + x[:1], y))) / 2

# Define your function
def process_file(file_path, parent_directory):

      predict_folder_path = os.path.join(os.path.dirname(file_path), 'predict')
      if os.path.exists(predict_folder_path) and os.path.isdir(predict_folder_path):
          print(f"Skipping {file_path} as it has a 'predict' folder.")
          return

      ID = []
      Names = []
      XYXY = []
      Masks = []
      Num_Frames = []
      n = []
      Class = []

      results = model.track(file_path,tracker="/content/ultralytics/ultralytics/cfg/trackers/botsort.yaml", project = file_dir, save_frames = False, save=True, persist = True, iou = 0.7, conf = 0.5, stream = True, save_conf = True, show_conf = False, show_labels = True, )
      for results in results:
        boxes = results.boxes
        masks = results.masks
        probs = results.probs
        id = results.boxes.id
        n.append(len(boxes))
        for i in range(len(results)):
          for x in range(len(results[i].boxes.xyxy.cpu().numpy())):
            try:
              if results.boxes is not None:
                ID.append(results[i][x].boxes.id.cpu().numpy())
              else:
                ID.append(None)
              if XYXY is not None:
                XYXY.append(results[i][x].boxes.xyxy.cpu().numpy())
              else:
                pass
              if Masks is not None:
                Masks.append(results[i][x].masks.xy)
              else:
                pass
              if Names is not None:
                Names.append(results[i][x].names)
              else:
                pass
            except AttributeError:
              pass
      #Set the analyzed video as a photo frame to draw on:
      algimg = results[0].orig_img
                          #______________________________________________________________________#____________________________________________________#Extract general information from ML algorithm:
      df = pd.DataFrame(ID, columns = ["ID"])
      df71234848 = np.vstack(Names)
      df71234848 = pd.DataFrame(df71234848, columns = ['Object_Name'])
      df2 = np.vstack(XYXY)
      df3 = pd.DataFrame(df2, columns = ("X1","Y1","X2","Y2"))
      df5 = pd.concat([df,df3], axis = 1)
      df5432 = pd.DataFrame(Masks, columns = ["Masks0"])

      # df5 = pd.concat([df,df3,df5432], axis = 1)
      # If used segmentation model ^Un-hash this to add Mask Coordinates to your DataFrame, be careful as highly dimensional objects may cause issues in your output .csv file

      df5 = pd.concat([df71234848, df5], axis = 1)

      #______________________________________________________________________#____________________________________________________#Assign Frame Numbers:

      df5['Frame_num'] = df5.groupby(['ID']).cumcount()+1

      #______________________________________________________________________#____________________________________________________#: Distance away from point q
      #Object_ID centroid coordinate:
      Centroid = []
      for x in range(len(df5)):
        Centroid.append([((df5['X1'][x]+df5['X2'][x])/2),((df5['Y1'][x]+df5['Y2'][x])/2)])

      Centroid_df = pd.DataFrame(Centroid, columns = ['Centroid_X','Centroid_Y'])
      df5 = pd.concat([df5,Centroid_df],axis=1)

      #Object_ID distance away from point q:

      distance = []
      if "q" in locals():
        for j in range(len(df5['Centroid_X'])):
          distance.append(math.sqrt( ((df5['Centroid_X'][j]-q[0])**2)+((df5['Centroid_Y'][j]-q[1])**2)))
        distance_df = pd.DataFrame(distance, columns = ['ID_Q_Distance'])
        df5 = pd.concat([df5,distance_df], axis = 1)


      #______________________________________________________________________#____________________________________________________#: Distance away from initial position: (FiX THE LOOP)

      #Adding Origin Distance to the correct ID:
      IDD_LIST = df5.ID.unique()
      Fly_ID_list = []
      Fly_ID_list.append(df5[df5['Frame_num'] == 1])
      Fly_ID_df = df5[['ID','Frame_num','Centroid_X','Centroid_Y']]
      Fly_ID_df = Fly_ID_df[Fly_ID_df['Frame_num'] == 1]
      Org_X = Fly_ID_df.Centroid_X.values.tolist()
      Org_Y = Fly_ID_df.Centroid_Y.values.tolist()
      centroid_origin = [(x, y) for x, y in zip(Org_X, Org_Y)]

      origin_index = 0
      for h in df5.ID.unique():
          fly_df = df5[df5['ID'] == h]
          for index, row in fly_df.iterrows():
              df5.loc[index, 'Origin_X'] = centroid_origin[origin_index][0]
              df5.loc[index, 'Origin_Y'] = centroid_origin[origin_index][1]
          origin_index += 1

      orig_distance = []
      for j in range(len(df5['Centroid_X'])):
        orig_distance.append(math.sqrt( ((df5['Centroid_X'][j]-df5['Origin_X'][j])**2)+((df5['Centroid_Y'][j]-df5['Origin_Y'][j])**2)))
      orig_distance_df = pd.DataFrame(orig_distance, columns = ['ID_Orig_Distance'])
      df5 = pd.concat([df5,orig_distance_df], axis = 1)

      #______________________________________________________________________#____________________________________________________#: Total Walk Distance by ID / Total Walk Distance by Frame:

      df10293847123948712349812374981234 = pd.DataFrame()
      df5 = df5.sort_values(by = ['ID','Frame_num','Centroid_X'])
      df5['Centroid_X_diff'] = abs(df5.groupby(['ID'])['Centroid_X'].diff().fillna(0))
      df5['Centroid_Y_diff'] = abs(df5.groupby(['ID'])['Centroid_Y'].diff().fillna(0))
      df5['Total_Centroid_diff'] = (df5['Centroid_X_diff'] + df5['Centroid_Y_diff'])
      df5['Total_Centroid_diff_cm'] = (df5['Total_Centroid_diff']/pixels_per_centimeter)
      df10293847123948712349812374981234['Centroid_X_total_diff'] = df5.groupby(['ID'])['Centroid_X_diff'].sum()
      df10293847123948712349812374981234['Centroid_Y_total_diff'] = df5.groupby(['ID'])['Centroid_Y_diff'].sum()
      df10293847123948712349812374981234['Total_Diff'] = df10293847123948712349812374981234['Centroid_X_total_diff'] + df10293847123948712349812374981234['Centroid_Y_total_diff']
      df10293847123948712349812374981234['Total_Diff_cm'] = (df10293847123948712349812374981234['Total_Diff'])/pixels_per_centimeter

      #______________________________________________________________________#____________________________________________________#: Surface area:
      #Calculating Surface area of an ID over timeframe _x_:
      Total_Area = []
      for u in range(len(df5432['Masks0'])):
        xy = df5432['Masks0'][u]
        x, y = zip(*xy)
        area = area_by_shoelace(x, y)
        Total_Area.append(area)
      df84646 = pd.DataFrame(Total_Area, columns = ['Total_Area'])
      df5 = pd.concat([df5,df84646],axis=1)

      Total_Area_Cm2 = []
      for h in range(len(df5['Total_Area'])):
        cm2 = (df5['Total_Area'][h])/(pixels_per_centimeter*pixels_per_centimeter)
        Total_Area_Cm2.append(cm2)

      df498149148 = pd.DataFrame(Total_Area_Cm2, columns = ['Total_Area_cm^2'])
      df5 = pd.concat([df5,df498149148],axis=1)


      #______________________________________________________________________#____________________________________________________#: Counting # of Individuals in each of the Polygons for Each Frame:

      Polygon1 = []
      Polygon2 = []
      Polygon3 = []
      if "coords1" in locals():
        for p in range(len(df5)):
          point = Point(df5['Centroid_X'][p],df5['Centroid_Y'][p])
          polygon1 = Polygon(coords1)
          Polygon1.append((polygon1.contains(point)))
        dfpolygon1 = pd.DataFrame(Polygon1, columns = ['Polygon1'])
        df5 = pd.concat([df5,dfpolygon1], axis = 1)

      if "coords2" in locals():
        for p in range(len(df5)):
          point = Point(df5['Centroid_X'][p],df5['Centroid_Y'][p])
          polygon2 = Polygon(coords2)
          Polygon2.append((polygon2.contains(point)))
        dfpolygon2 = pd.DataFrame(Polygon2, columns = ['Polygon2'])
        df5 = pd.concat([df5,dfpolygon2], axis = 1)

      if "coords3" in locals():
        for p in range(len(df5)):
          point = Point(df5['Centroid_X'][p],df5['Centroid_Y'][p])
          polygon3 = Polygon(coords3)
          Polygon3.append((polygon3.contains(point)))
        dfpolygon3 = pd.DataFrame(Polygon3, columns = ['Polygon3'])
        df5 = pd.concat([df5,dfpolygon3], axis = 1)


      #______________________________________________________________________#____________________________________________________#: Counting Individuals:

      # df456789 = pd.DataFrame(n, columns = ['Object_Counts'])

      #______________________________________________________________________#____________________________________________________#: CHILL COMA RECOVERY:

      #Create the data frame for movement over max_n frames:

      max_n = 1800  # Rolling Window Frame
      threshold = 700 # Locomotor Activity Threshold (pixels)

      df5['Cumulative_Sum_Of_Movement'] = df5.groupby('ID')['Total_Centroid_diff'].transform(lambda x: x.rolling(window=max_n, min_periods=1).sum())
      df5['Movement_Threshold'] = (df5['Cumulative_Sum_Of_Movement'] >= threshold).astype(int)

      #Filter and data-frame the data to see what is the earliest frame at which certain motility threshold has been reached:

      filtered_df5 = df5[df5['Movement_Threshold'] == 1]
      CCR_df = filtered_df5.groupby('ID')['Frame_num'].min().reset_index()
      print(CCR_df)

      #Remove any columns under a certain frame threshold:
      #df5 = df5.groupby('ID').filter(lambda x : len(x) > 500)

      #Expose the dataframe:
      df5.sort_values('ID')
      print(df5.groupby(['ID']).count())

      #Expose the dataframe:
      df10293847123948712349812374981234.sort_values('ID')
      print(df10293847123948712349812374981234.groupby(['ID']).count())

      video_filename = os.path.basename(file_path)
      video_name, _ = os.path.splitext(video_filename)
      Total_csv_file_path = os.path.join(root, f'{video_name}_Total.csv')
      Net_csv_file_path = os.path.join(root, f'{video_name}_Net.csv')

      #Total_csv_file_path = os.path.join(root, f'{parent_directory}_Total.csv')
      #Net_csv_file_path = os.path.join(root, f'{parent_directory}_Net.csv')

      #Save the _main_dataframe:
      df5.to_csv(Total_csv_file_path)

      #Save the _net_dataframe:
      df10293847123948712349812374981234.to_csv(Net_csv_file_path)

      #Save _Object_Count DataFrame:
      # df456789.to_csv('_object_count_filename.csv')

      #Save CCR DataFrame:
      # CCR_df.to_csv('/content/drive/MyDrive/ChillComaRecovery/TheOneIPracticed/Data/_CCR_.csv')


def get_random_frame(folder_path):
    # List all video files in the folder (common video formats)
    video_extensions = [".mp4", ".avi", ".mov", ".mkv", ".flv", ".wmv", ".webm", ".MOV", ".MP4",".AVI"]
    video_files = [file for file in os.listdir(folder_path) if any(file.endswith(ext) for ext in video_extensions)]

    if not video_files:
        print("No video files found in the folder.")
        return None

    # Choose a random video file
    random_video_file = random.choice(video_files)

    # Path to the chosen video file
    video_file_path = os.path.join(folder_path, random_video_file)

    # Open the video file
    cap = cv2.VideoCapture(video_file_path)

    if not cap.isOpened():
        print(f"Error opening video file: {video_file_path}")
        return None

    # Get total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Choose a random frame index
    random_frame_index = random.randint(0, total_frames - 1)

    # Set the frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES, random_frame_index)

    # Read the frame
    ret, frame = cap.read()

    # Check if the frame is read correctly
    if ret:
        # Convert frame to RGB (OpenCV reads in BGR)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return frame_rgb
    else:
        print(f"Error reading frame {random_frame_index} from {random_video_file}")
        return None

display.clear_output()

# **MODEL SELECTION AND TRAINING**

#### **Upload your Roboflow code and install your annotated data**

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Test Code")
project = rf.workspace("Test Code").project("Test Code")
dataset = project.version(0).download("Test Code")

### **Train your model**

In [ ]:
#Select model & Train it
modell = YOLO('Model of Your Choice')
modell.train(data = "Insert Your Data.yaml in your training-data-set folder", epochs = 100, imgsz= [w, h], batch = 5, project = "Directory to Output the Model")

# **Automatic Video Analysis:**

____

### **Calculate pixel per centimeter metric for MLDAAPP conversions**

In [ ]:
#Define your video directory:
directory_path = 'Directory/To/Your/Videos'
image = get_random_frame(directory_path)

#Draw a line to calculate the pixel per centimeter variable for MLDAAPP conversions
draw_lines(image,0,0,0,0) #x1,y1,x2,y2

### **Run Video Analysis:**

In [ ]:
model = YOLO("Directory/To/best.pt")
pixels_per_centimeter = 144 #IMPORTANT: Define this variable from your own calculation

directory_path = 'Directory/To/Your/Videos'

# Recursively search for .MOV files
for root, dirs, files in os.walk(directory_path):
    for filename in files:
        if filename.endswith('.MOV'):  #IMPORTANT: Modify .MOV with your video format
            # Get the full path to the file
            file_path = os.path.join(root, filename)

            file_dir = os.path.dirname(file_path)

            # Get the name of the parent directory
            parent_directory = os.path.basename(root)

            process_file(file_path, parent_directory)

# **Photo Analysis:**

---



## **Run the model on your data and save to .csv**

In [ ]:
#Select your Custom Trained Model:
model = YOLO("Your best.pt file directory")

#Run the model to extract the counts from each photo.
n = []
results = model.predict("Your Photo Data Folder", save = True, conf = 0.32, show_labels=False) #IMPORTANT: Change Your Photo Data Folder to your Photo folder directory!

for result in results:
  boxes = result.boxes
  masks = result.masks
  keypoints = result.keypoints
  probs = result.probs
  n.append(len(boxes))

#Save your data to a .csv file
save_to_csv("FileName.csv", "ColumnName", n) #IMPORTANT: Change the name opf FIleName.csv to your preferred file name, and change ColumnName to your preferred column name.

# **Manual Video Analysis:**

---



###**VIDEO EDITING**

In [ ]:
video_editing("Video Directory",t0 = 0, t1=10, fps = 30, spd=1, w = 1920)
frame = global_video

### **Model Usage**

In [ ]:
#Set your video directory as frame variable if video_editing was not used:

# frame = 'Test Video Directory'

#Set empty data arrays to import model extracted data into:
ID = []
Names = []
XYXY = []
Masks = []
Num_Frames = []
n = []
Class = []

results = model.track(frame,tracker="/content/ultralytics/ultralytics/cfg/trackers/botsort.yaml",save=True, persist = True, iou = 0.4, conf = 0.4, stream = True, save_conf = True)
for results in results:
  boxes = results.boxes
  masks = results.masks
  probs = results.probs
  id = results.boxes.id
  n.append(len(boxes))
  for i in range(len(results)):
    for x in range(len(results[i].boxes.xyxy.cpu().numpy())):
      if ID is not None:
        ID.append(results[i][x].boxes.id.cpu().numpy())
      else:
        pass
      if XYXY is not None:
        XYXY.append(results[i][x].boxes.xyxy.cpu().numpy())
      else:
        pass
      if Masks is not None:
        Masks.append(results[i][x].masks.xy)
      else:
        pass
      if Names is not None:
        Names.append(results[i][x].names)
      else:
        pass

#Set the analyzed video as a photo frame to draw on:
algimg = results[0].orig_img


### **Drawing on the image**

Run this block even if not needed, it feeds neccessary data to process Data Extraction block.

In [ ]:
draw_dots(algimg, 874,848) #keep algimg
#                  x1  y1
draw_lines(algimg,200,400,600,800) #keep algimg
#                 x1 y1 x2 y2
draw_polygons(algimg, #keep algimg
              100,200,  #x1 y1
              400,500,  #x2 y2
              600,700,  #x3 y3
              800,900)  #x4 y4
# coords1 = [[100,200],[400,500],[600,700],[800,900]]
#           x1  y1    x2  y2    x3  y3    x4  y4
draw_polygons(algimg, #keep algimg
              100,200,  #x1 y1
              400,500,  #x2 y2
              600,700,  #x3 y3
              800,900)  #x4 y4
# coords2 = [[101,201],[400,501],[600,701],[801,901]]
#           x1  y1    x2  y2    x3  y3    x4  y4
#IMPORTANT:
#Set the conversions from pixeles >>> cm:
pixels_per_centimeter = 144

#Set the custom q point within space (recommend using draw_dots):
# q = []

### **Data Extraction:**

In [ ]:
#______________________________________________________________________#____________________________________________________#Extract general information from ML algorithm:
df = pd.DataFrame(ID, columns = ["ID"])
df71234848 = np.vstack(Names)
df71234848 = pd.DataFrame(df71234848, columns = ['Object_Name'])
df2 = np.vstack(XYXY)
df3 = pd.DataFrame(df2, columns = ("X1","Y1","X2","Y2"))
df5 = pd.concat([df,df3], axis = 1)
df5432 = pd.DataFrame(Masks, columns = ["Masks0"])

# df5 = pd.concat([df,df3,df5432], axis = 1)
# If used segmentation model ^Un-hash this to add Mask Coordinates to your DataFrame, be careful as highly dimensional objects may cause issues in your output .csv file

df5 = pd.concat([df71234848, df5], axis = 1)

#_________________________________________________________________#____________________________________________________#Reassign IDs for _main_ dataframe:

# #Remove uneeded columns/ids: (Always re-assign higher # > lower #)

# Rename column #A into column #B:
# df5['ID'] = df5['ID'].replace([465],[455])
# df5['ID'] = df5['ID'].replace([375],[346])


#______________________________________________________________________#____________________________________________________#Assign Frame Numbers:

df5['Frame_num'] = df5.groupby(['ID']).cumcount()+1

#______________________________________________________________________#____________________________________________________#: Distance away from point q
#Object_ID centroid coordinate:
Centroid = []
for x in range(len(df5)):
  Centroid.append([((df5['X1'][x]+df5['X2'][x])/2),((df5['Y1'][x]+df5['Y2'][x])/2)])

Centroid_df = pd.DataFrame(Centroid, columns = ['Centroid_X','Centroid_Y'])
df5 = pd.concat([df5,Centroid_df],axis=1)

#Object_ID distance away from point q:

distance = []
if "q" in locals():
  for j in range(len(df5['Centroid_X'])):
    distance.append(math.sqrt( ((df5['Centroid_X'][j]-q[0])**2)+((df5['Centroid_Y'][j]-q[1])**2)))
  distance_df = pd.DataFrame(distance, columns = ['ID_Q_Distance'])
  df5 = pd.concat([df5,distance_df], axis = 1)


#______________________________________________________________________#____________________________________________________#: Distance away from initial position:

#Adding Origin Distance to the correct ID:
IDD_LIST = df5.ID.unique()
Fly_ID_list = []
Fly_ID_list.append(df5[df5['Frame_num'] == 1])
Fly_ID_df = df5[['ID','Frame_num','Centroid_X','Centroid_Y']]
Fly_ID_df = Fly_ID_df[Fly_ID_df['Frame_num'] == 1]
Org_X = Fly_ID_df.Centroid_X.values.tolist()
Org_Y = Fly_ID_df.Centroid_Y.values.tolist()
centroid_origin = [(x, y) for x, y in zip(Org_X, Org_Y)]

origin_index = 0
for h in df5.ID.unique():
    fly_df = df5[df5['ID'] == h]
    for index, row in fly_df.iterrows():
        df5.loc[index, 'Origin_X'] = centroid_origin[origin_index][0]
        df5.loc[index, 'Origin_Y'] = centroid_origin[origin_index][1]
    origin_index += 1

orig_distance = []
for j in range(len(df5['Centroid_X'])):
  orig_distance.append(math.sqrt( ((df5['Centroid_X'][j]-df5['Origin_X'][j])**2)+((df5['Centroid_Y'][j]-df5['Origin_Y'][j])**2)))
orig_distance_df = pd.DataFrame(orig_distance, columns = ['ID_Orig_Distance'])
df5 = pd.concat([df5,orig_distance_df], axis = 1)

#______________________________________________________________________#____________________________________________________#: Total Walk Distance by ID / Total Walk Distance by Frame:

df10293847123948712349812374981234 = pd.DataFrame()
df5 = df5.sort_values(by = ['ID','Frame_num','Centroid_X'])
df5['Centroid_X_diff'] = abs(df5.groupby(['ID'])['Centroid_X'].diff().fillna(0))
df5['Centroid_Y_diff'] = abs(df5.groupby(['ID'])['Centroid_Y'].diff().fillna(0))
df5['Total_Centroid_diff'] = (df5['Centroid_X_diff'] + df5['Centroid_Y_diff'])
df5['Total_Centroid_diff_cm'] = (df5['Total_Centroid_diff']/pixels_per_centimeter)
df10293847123948712349812374981234['Centroid_X_total_diff'] = df5.groupby(['ID'])['Centroid_X_diff'].sum()
df10293847123948712349812374981234['Centroid_Y_total_diff'] = df5.groupby(['ID'])['Centroid_Y_diff'].sum()
df10293847123948712349812374981234['Total_Diff'] = df10293847123948712349812374981234['Centroid_X_total_diff'] + df10293847123948712349812374981234['Centroid_Y_total_diff']
df10293847123948712349812374981234['Total_Diff_cm'] = (df10293847123948712349812374981234['Total_Diff'])/pixels_per_centimeter

#______________________________________________________________________#____________________________________________________#: Surface area:
#Calculating Surface area of an ID over timeframe _x_:
Total_Area = []
for u in range(len(df5432['Masks0'])):
  xy = df5432['Masks0'][u]
  x, y = zip(*xy)
  area = area_by_shoelace(x, y)
  Total_Area.append(area)
df84646 = pd.DataFrame(Total_Area, columns = ['Total_Area'])
df5 = pd.concat([df5,df84646],axis=1)

Total_Area_Cm2 = []
for h in range(len(df5['Total_Area'])):
  cm2 = (df5['Total_Area'][h])/(pixels_per_centimeter*pixels_per_centimeter)
  Total_Area_Cm2.append(cm2)

df498149148 = pd.DataFrame(Total_Area_Cm2, columns = ['Total_Area_cm^2'])
df5 = pd.concat([df5,df498149148],axis=1)


#______________________________________________________________________#____________________________________________________#: Counting # of Individuals in each of the Polygons for Each Frame:

Polygon1 = []
Polygon2 = []
Polygon3 = []
if "coords1" in locals():
  for p in range(len(df5)):
    point = Point(df5['Centroid_X'][p],df5['Centroid_Y'][p])
    polygon1 = Polygon(coords1)
    Polygon1.append((polygon1.contains(point)))
  dfpolygon1 = pd.DataFrame(Polygon1, columns = ['Polygon1'])
  df5 = pd.concat([df5,dfpolygon1], axis = 1)

if "coords2" in locals():
  for p in range(len(df5)):
    point = Point(df5['Centroid_X'][p],df5['Centroid_Y'][p])
    polygon2 = Polygon(coords2)
    Polygon2.append((polygon2.contains(point)))
  dfpolygon2 = pd.DataFrame(Polygon2, columns = ['Polygon2'])
  df5 = pd.concat([df5,dfpolygon2], axis = 1)

if "coords3" in locals():
  for p in range(len(df5)):
    point = Point(df5['Centroid_X'][p],df5['Centroid_Y'][p])
    polygon3 = Polygon(coords3)
    Polygon3.append((polygon3.contains(point)))
  dfpolygon3 = pd.DataFrame(Polygon3, columns = ['Polygon3'])
  df5 = pd.concat([df5,dfpolygon3], axis = 1)


#______________________________________________________________________#____________________________________________________#: Counting Individuals:
# df456789 = pd.DataFrame(n, columns = ['Object_Counts'])



#_______________________________________________________Remove any specific ID
# df5 = df5[df5['ID'] != 417] #Remove any specific ID



#______________________________________________________Remove any columns under a certain frame threshold:
# df5 = df5.groupby('ID').filter(lambda x : len(x) > 500)



#_____________________Expose the dataframe:
df5.sort_values('ID')
print(df5.groupby(['ID']).count())

#___________________________________________________Expose the dataframe:
df10293847123948712349812374981234.sort_values('ID')
print(df10293847123948712349812374981234.groupby(['ID']).count())

### **Save the DataFrames to .csv Files:**

In [ ]:
#Save the _main_dataframe:
df5.to_csv('directory/_main_filename.csv')

#Save the _net_dataframe:
df10293847123948712349812374981234.to_csv('directory/_net_filename.csv')

#Save _Object_Count DataFrame:
df456789.to_csv('directory/_object_count_filename.csv')